## Nossa missão ##

Detecção de spam é uma das principais aplicações de Machine Learning nas interwebs hoje em dia. Praticamente todos principais provedores de serviço de e-mail têm sistemas de detecção de spam e classificam esse tipo de mensagem como "lixo eletrônico".

Nesta missão vamos usar o algoritmo ingênuo de Bayes para criar um modelo que seja capaz de classificar mensagens SMS em um [dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) como spam ou não-spam, de acordo com o treinamento que aplicarmos ao modelo. É importante ter um certo nível de intuição sobre a cara típica do spam em mensagens de texto. Normalmente estas mensagens contém palavras como "gratuito", "ganhe", "ganhador", "dinheiro", "prêmio" e coisas do tipo, uma vez que elas são projetadas para chamar sua atenção e, de alguma forma, te instigar a abrí-las. Além disso, mensagens de spam tendem a ter várias palavras escritas apenas em letras maiúsculas e muitos pontos de exclamação. Para o destinatário, normalmente é bem simples de se identificar uma mensagem de spam e nosso objetivo aqui é o de treinar um modelo para fazer isso para nós!

Ser capaz de identificar mensagens de spam é um problema de classificação binária, uma vez que as mensagens são classificadas como "spam" ou "não-spam" e nada mais. Ademais, trata-se de um problema de aprendizado supervisionado, uma vez que iremos apresentar um dataset rotulado para o modelo, a partir do qual ele poderá aprender e fazer previsões futuras. 

### Passo 0: Introdução ao Teorema Ingênuo de Bayes ###

O teorema de Bayes é um dos algoritmos de inferência probabilística mais antigos, criado pelo Reverendo Bayes (que o usou para tentar inferir nada menos que a existência de Deus) e que ainda funciona muito bem para certos casos de uso.

É mais fácil de se entender este teorema usando um exemplo. Digamos que você fosse um membro do Serviço Secreto que foi designado para proteger o/a candidato/a Democrata à presidência durante um dos seus discursos. Como se trata de um evento aberto ao público, seu trabalho não seria fácil e você precisaria estar constantemente atento a ameaças. Uma forma de começar esta avaliação seria atribuir um fator de risco para cada pessoa. Então, a partir de características de um indivíduo, como idade, gênero e sua resposta a perguntas como "a pessoa está carregando uma bolsa?", "a pessoa parece nervosa?", etc., você poderia julgar se aquela pessoa apresenta uma ameaça viável.

Caso um indivíduo atenda aos critérios escolhidos a ponto de levantar suspeitas, você pode agir e remover aquela pessoa da vizinhança do evento. O teorema de Bayes funciona da mesma forma, uma vez que computamos a probabilidade de um evento (uma pessoa apresentar uma ameaça) baseado na probabilidade de certos eventos relacionados (idade, gênero, presença de bolsa ou não, nervosismo da pessoa, etc.).

Uma coisa a ser considerada é a independência destes fatores entre si. Por exemplo, caso uma criança pareça nervosa no evento, a chance dela apresentar uma ameaça não é tão grande quanto a de um adulto nervoso. Para detalhar um pouco mais este ponto, aqui estamos considerando duas características: idade E nervosismo. Suponha que estejamos olhando para cada uma destas características individualmente, poderíamos projetar um modelo que sinaliza TODAS as pessoas que estiverem nervosas como ameaças em potencial. Entretanto, é provável que tenhamos vários falsos positivos, uma vez que existe uma alta chance que crianças presentes no evento estejam nervosos. Assim, ao considerar a idade de um indivíduo conjuntamente ao nível de nervosismo, certamente teríamos um resultado mais preciso sobre quem apresenta um maior ou menor potencial de ameaça.

Esta á a parte "ingênua" do teorema - ele considera que todas características são independentes das demais, embora isto não seja sempre verdade. Isto pode afetar o julgamento final apresentado pelo algoritmo.

Resumindo, o teorema de Bayes calcula a probabilidade que um certo evento aconteça (no nosso caso, uma mensagem ser spam) baseado nas distribuições de probabilidade conjuntas de outros eventos determinados (no nosso caso, uma mensagem ser classificada como spam). Vamos mergulhar no funcionamento do teorema de Bayes mais tarde nesta missão, mas antes vamos entender os dados com que iremos trabalhar.

### Passo 1.1: Entendendo nosso dataset ### 


Vamos usar um [dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) do repositório de Machine Learning da UCI, que tem uma coleção muito boa de datasets para propósitos de pesquisa experimental. O link direto para baixar os dados está [aqui](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/).


 ** Aqui vai uma prévia dos dados: ** 

<img src="images/dqnb.png" height="1242" width="1242">

As colunas do dataset não foram nomeadas e, como você pode ver, existem 2 colunas. 

A primeira coluna recebe dois valores: "ham", que significa que a mensagem não é spam, e "spam", que significa que a mensagem é spam.

A segunda coluna apresenta o conteúdo em texto da mensagem SMS que está sendo classificada.

>** Instruções: **
* Importe o dataset para um dataframe pandas usando o método read_table. Como se trata de um dataset separado por tab, usaremos '\t' como o valor do argumento 'sep' que especifica este formato. 
* Além disso, renomeie os nomes das colunas especificando uma lista ['label, 'sms_message'] como o argumento 'names' de read_table().
* Exiba os cinco primeiros valores do dataframe com os novos nomes das colunas.

In [ ]:
'''
Solução
'''
import pandas as pd
# Dataset obtido em - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = pd.read_table('smsspamcollection/SMSSpamCollection',
                   sep='\t', 
                   header=None, 
                   names=['label', 'sms_message'])

# Saída exibindo os 5 primeiros valores
df.head()

### Passo 1.2: Pré-processamento de dados ###

Agora que entendemos como nosso dataset está organizado, vamos converter nossos rótulos para variáveis binárias, 0 para representar 'ham' (i.e. não-spam) e 1 para representar 'spam' para facilitar a computação.

Você pode estar se perguntando: por que precisarmos deste passo? A resposta está na forma como o scikit-learn lida com entradas. O scikit-learn lida apenas com valores numéricos. Portanto, se deixássemos nossos rótulos como strings, o scikit-learn faria a conversão internamente (mais especificamente, os rótulos string seriam convertidos para valores float desconhecidos). 

Nosso modelo ainda poderia fazer previsões caso deixássemos nossos rótulos como strings, mas poderíamos ter problemas mais tarde ao calcular métricas de desempenho como, por exemplo, escores de precisão e recall. Assim, para evitar "peguinhas" inesperados mais tarde, é uma boa prática fazer com que nossos valores categóricos sejam entregues para nosso modelo como números inteiros.

>**Instruções: **
* Converta os valores na coluna 'label' para valores numéricos usando o método de mapeamento, como visto a seguir:
{'ham':0, 'spam':1} Isto associa o valor 'ham' ao 0 e o valor 'spam' ao 1.
* Além disso, para ter uma ideia do tamanho do dataset com que estamos lidando, exiba na tela o número de linhas e colunas usando 
'shape'.

In [ ]:
'''
Solução
'''
df['label'] = df.label.map({'ham':0, 'spam':1})
print(df.shape)
df.head() # retorna (linhas, colunas)

### Passo 2.1: Sacola de palavras ###

O que temos aqui no nosso dataset é uma grande coleção de dados de texto (5.572 linhas de dados). A maior parte dos algoritmos de ML conta com que entradas numéricas sejam apresentadas, mas mensagens de e-mail/SMS normalmente contém bastante texto.

Aqui gostaríamos de introduzir o conceito de Bag of Words (BoW), que é um termo usado para especificar problemas que contém uma "sacola de palavras" ou uma coleção de dados de texto que precisam ser processados. A ideia básica do BoW é pegar um pedaço de texto e contar a frequência das palavras naquele texto. É importante notar que, no conceito de BoW, cada palavra é considerada individualmente e a ordem de ocorrência das palavras não importa.  
Usando um processo que explicaremos agora, podemos converter uma coleção de documentos para uma matriz, em que cada linha representa um documento e cada coluna representa uma palavra (token), de modo que cada coordenada (linha,coluna) da matriz contenha a contagem da frequência de ocorrência de uma determinada palavra ou token em um documento específico.

Por exemplo: 

Digamos que temos as seguintes 4 mensagens:

`['Hello, how are you!',
'Win money, win from home.',
'Call me now',
'Hello, Call you tomorrow?']`

Nosso objetivo aqui é converter este conjunto de mensagens de texto em uma matriz de distribuição de frequências, como a seguir:

<img src="images/countvectorizer.png" height="542" width="542">

Aqui, como podemos ver, as mensagens são numeradas nas linhas e as palavras são representadas por colunas, com o valor correspondente sendo a frequência daquela palavra na mensagem.

Vamos analisar este processo passo a passo e ver como podemos fazer esta conversão usando um conjunto pequeno de mensagens.

Para lidar com isso usaremos o método [count vectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) do sklearn, que faz o seguinte:

* Tokeniza a string (separa a string em palavras) e associa um ID inteiro a cada token.
* Conta a frequência de ocorrência de cada um destes tokens.

** Observação: ** 

* O método CountVectorizer converte todas palavras tokenizadas automaticamente para letras minúsculas, para que ele não trate palavras como 'Ele' e 'ele' diferentemente. Isto pode ser configurado através do parâmetro `lowercase`, cujo valor padrão é `True`.

* Ele também ignora toda pontuação, de modo que palavras seguidas por um sinal de pontuação (por exemplo: 'olá!') não são tratadas de forma diferente das mesmas palavras sem pontuação antes ou depois (por exemplo: 'olá'). Isto é feito através do parâmetro `token_pattern`, cujo valor padrão é uma expressão regular que seleciona tokens de 2 ou mais caracteres alfanuméricos.

* O terceiro parâmetro que deve ser observado é o parâmetro `stop_words`. Stop words (conhecidas como "palavras vazias" em português) se referem às palavras mais comumente usadas em uma língua. Elas incluem palavras como "sou", "um", "e", "o", etc. Ao configurar este parâmetro como `english`, CountVectorizer ignorará automaticamente todas as palavras (presentes no texto de entrada) que forem encontradas na lista de palavras vazias do inglês embutida no scikit-learn. Isto é extremamente útil, uma vez que palavras vazias podem enviesar nossos cálculos quando estamos tentando encontrar certas palavras-chaves indicativas de spam.

Vamos nos aprofundar na aplicação de cada um destes parâmetros no nosso modelo em um passo futuro, mas por hora é importante apenas estar ciente que tais técnicas de pré-processamento estão disponíveis quando lidamos com dados textuais.

### Passo 2.2: Implementando Bag of Words do zero ###

Antes de mergulharmos na biblioteca de Bag of Words (BoW) do scikit-learn, para que ela faça o trabalho sujo por nós, vamos implementar o algoritmo por conta própria para que possamos entender o que está acontecendo nos bastidores. 

** Passo 1: Converter todas string para letras minúsculas. **

Digamos que temos este conjunto de documentos:

```
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']
```
>>** Instruções: **
* Converta todas strings no conjunto de documentos para letras minúsculas. Salve o resultado em uma lista chamada de 'lower_case_documents'. Você pode converter strings para minúsculas em python usando o método lower().


In [ ]:
'''
Solução:
'''
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = []
for i in documents:
    lower_case_documents.append(i.lower())
print(lower_case_documents)

** Passo 2: Remover toda pontuação **

>>**Instruções: **
Remova toda pontuação das strings no conjunto de documentos. Salve o resultado em uma lista chamada de 'sans_punctuation_documents'. 

In [ ]:
'''
Solução:
'''
sans_punctuation_documents = []
import string

for i in lower_case_documents:
    sans_punctuation_documents.append(i.translate(str.maketrans('', '', string.punctuation)))
print(sans_punctuation_documents)

** Passo 3: Tokenizar **

Tokenizar uma frase em um conjunto de documentos significa dividir uma frase em palavras usando um delimitador. O delimitador especifica que caractere usaremos para identificar o início e fim de uma palavra (por exemplo, poderíamos usar um espaço simples como delimitador para identificação de palavras no nosso conjunto de palavras).

>>**Instrucções:**
Tokenize as strings armazenadas em 'sans_punctuation_documents' usando o método split() e armazene o conjunto de documentos final em uma lista chamada de 'preprocessed_documents'.


In [ ]:
'''
Solução:
'''
preprocessed_documents = []
for i in sans_punctuation_documents:
    preprocessed_documents.append(i.split(' '))
print(preprocessed_documents)

** Passo 4: Contar frequências **

Agora que organizamos nosso conjunto de documentos no formato exigido, podemos começar a contar a ocorrência de cada palavra em cada mensagem do conjunto de documentos. Usaremos o método `Counter` da biblioteca `collections` do Python para isto. 

`Counter` conta a ocorrência de cada item de uma lista e retorna um dicionário, com a chave sendo o item que foi contado e com o valor correspondente sendo a contagem daquele item na lista. 

>>**Instruções:**
Usando o método Counter() e a lista preprocessed_documents como entrada, crie um dicionário com as chaves sendo cada palavra em cada documento e cada valor correspondente sendo a frequência de ocorrência daquela palavra. Salve cada dicionário obtido do método Counter como um item em uma lista chamada de 'frequency_list'.


In [ ]:
'''
Solução
'''
frequency_list = []
import pprint
from collections import Counter

for i in preprocessed_documents:
    frequency_counts = Counter(i)
    frequency_list.append(frequency_counts)
pprint.pprint(frequency_list)

Parabéns! Você implementou o processo Bag of Words do zero! Como podemos ver na nossa saída anterior, temos um dicionário de distribuição de frequências que nos dá uma visão clara do texto com que estamos lidando.

Agora deveríamos ter um entendimento sólido do que está acontecendo nos bastidores no método `sklearn.feature_extraction.text.CountVectorizer` do scikit-learn. 

Agora vamos implementar o método `sklearn.feature_extraction.text.CountVectorizer` no próximo passo.

### Passo 2.3: Implementando Bag of Words no scikit-learn ###

Agora que implementamos o conceito BoW do zero, vamos seguir em frente e usar o scikit-learn para fazer este processo de forma limpa e sucinta. Vamos usar o mesmo conjunto de documentos que usamos no passo anterior. 

In [ ]:
'''
Aqui vamos criar uma matriz de frequências em um conjunto de documentos menor para nos assegurarmos que entendemos como 
a geração de matrizes documento-termo acontece. Nós criamos um conjunto de documentos de exemplo chamado de 'documents'.
'''
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

>>**Instruções:**
Importe o método sklearn.feature_extraction.text.CountVectorizer e crie uma instância dele chamada de 'count_vector'. 

In [ ]:
'''
Solução
'''
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

** Pré-processamento de dados com CountVectorizer() **

No Passo 2.2, implementamos uma versão do método CountVectorizer() do zero que implicou, primeiramente, na limpeza dos nossos dados. Esta limpeza envolveu a conversão de todos dados para letras minúsculas e a remoção de todos sinais de pontuação. CountVectorizer() tem certos parâmetros que tomam conta destes passos para nós. São eles:

* `lowercase = True`
    
    O parâmetro `lowercase` tem um valor padrão de `True` que converte todo nosso texto para letras minúsculas.


* `token_pattern = (?u)\\b\\w\\w+\\b`
    
    O parâmetro `token_pattern` tem uma expressão regular padrão `(?u)\\b\\w\\w+\\b` que ignora todos sinais de pontuação e os trata como delimitadores, enquanto aceita strings alfanuméricas de comprimento maior ou igual a 2, como tokens individuais ou palavras.


* `stop_words`

    O parâmetro `stop_words`, se configurado para `english`, removerá todas as palavras do nosso conjunto de palavras que corresponderem a uma lista de palavras vazias do inglês que está definida no scikit-learn. Considerando o tamanho do nosso dataset e o fato que estamos lidando com mensagens SMS, e não fontes de texto mais longo como e-mail, não iremos configurar este parâmetro.

Você pode ver todos os valores de parâmetros do seu objeto `count_vector` simplesmente exibindo o objeto como a seguir:

In [ ]:
'''
Nó de prática:
Exiba o objeto 'count_vector', que é uma instância de 'CountVectorizer()'
'''
print(count_vector)

>>**Instruções:**
Ajuste seu dataset de documentos ao objeto CountVectorizer que você criou usando fit() e pegue a lista de palavras que foram categorizadas como características usando o método get_feature_names().

In [ ]:
'''
Solução:
'''
count_vector.fit(documents)
count_vector.get_feature_names()

O método `get_feature_names()` retorna os nomes das características para este dataset, que é o conjunto de palavras que compõem o vocabulário para 'documents'.

>>**
Instruções:**
Crie uma matriz em que as linhas representam cada um dos 4 documentos, e as colunas representam cada palavra. O valor correspondente (linha, coluna) é a frequência de ocorrência de uma palavra específica (da coluna) em um documento específico (da linha). Você pode fazer isso usando o método transform() e passando o dataset de documentos como argumento. O método transform() retorna uma matriz de inteiros do numpy, que você pode converter para um array usando toarray(). Chame o array de 'doc_array'


In [ ]:
'''
Solução
'''
doc_array = count_vector.transform(documents).toarray()
doc_array

Agora temos uma representação limpa dos documentos em termos da distribuição da frequência de palavras neles. Para facilitar o entendimento, nosso próximo passo é converter este array em um dataframe e nomear as colunas adequadamente.

>>**Instruções:**
Converta o array que obtivemos, carregado em 'doc_array', para um dataframe e configure os nomes das colunas para palavras (que você obteve anteriormente usando get_feature_names(). Chame este dataframe de 'frequency_matrix'.


In [ ]:
'''
Solução
'''
frequency_matrix = pd.DataFrame(doc_array, 
                                columns = count_vector.get_feature_names())
frequency_matrix

Parabéns! Você implementou com sucesso um problema Bag of Words para um dataset de documentos que criamos.

Um potencial problema que pode surgir ao se usar este método de primeira é o fato que, se nosso dataset de texto for extremamente grande (digamos que temos uma coleção enorme de artigos de notícias ou mensagens de e-mail), haverá certos valores que são muito mais comuns que outros simplesmente devido à estrutura da linguagem em si. Então palavras como 'é', 'o', 'um', pronomes, construções gramaticais, etc, poderiam enviesar nossa matriz e afetar nossa análise. 

Existem algumas formas de mitigar este problema. Uma forma é usar o parâmetro `stop_words` e configurá-lo para `english`. Isto fará com que todas as palavras (dos nossos documentos de entrada) que estiverem na lista de palavras vazias do inglês do scikit-learn sejam ignoradas.

Outra forma de mitigar isto é usar o método [tfidf](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Este método está fora de escopo no contexto desta lição.

### Passo 3.1: Conjuntos de treinamento e teste ###

Agora que entendemos como lidar com o problema Bag of Words, podemos voltar para nosso dataset e continuar nossa análise. Nosso primeiro passo nesse sentido seria dividir nosso dataset em conjuntos de treinamento e teste para que possamos testar nosso modelo mais tarde. 


>>**Instruções:**
Divida o dataset em conjuntos de treinamento e teste usando o método train_test_split do sklearn. Divida os dados usando as seguintes variáveis:
* `X_train` são nossos dados de treinamento para a coluna 'sms_message'.
* `y_train` são nossos dados de treinamento para a coluna 'label'
* `X_test` são nossos dados de teste para a coluna 'sms_message'.
* `y_test` são nossos dados de teste para a coluna 'label' column.
Exiba o número de linhas que temos em cada um dos dados de treinamento e teste.


In [ ]:
'''
Solução

OBSERVAÇÃO: sklearn.cross_validation será desativado e substituído por sklearn.model_selection em breve
'''
# divida em conjuntos de treinamento e teste
# USE from sklearn.model_selection import train_test_split para evitar avisos de desativação.
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], 
                                                    df['label'], 
                                                    random_state=1)

print('Número de linhas no dataset: {}'.format(df.shape[0]))
print('Número de linhas no conjunto de treinamento: {}'.format(X_train.shape[0]))
print('Número de linhas no conjunto de teste: {}'.format(X_test.shape[0]))

### Passo 3.2: Aplicando o processo Bag of Words ao nosso dataset. ###

Agora que dividimos os dados, nosso próximo objetivo é seguir os passos a partir do "Passo 2: Sacola de palavras" e converter nossos dados para o formato de matriz desejado. Para fazer isto, usaremos CountVectorizer() como fizemos anteriormente. Existem dois passos a considerar aqui:

* Inicialmente, precisamos ajustar nossos dados de treinamento (`X_train`) em `CountVectorizer()` e retornar a matriz.
* Em seguida, precisamos transformar nossos dados de teste (`X_test`) e retornar a matriz. 

Note que `X_train` são nossos dados de treinamento para a coluna 'sms_message' do nosso dataset e usaremos isto para treinar nosso modelo. 

`X_test` são nossos dados de teste da coluna 'sms_message' column e estes são os dados que usaremos (depois da transformação para matriz) para testar nossas previsões. Então iremos comparar estas previsões com `y_test` em um passo posterior. 

Por enquanto, fornecemos o código que faz as transformações para matriz para você!

In [ ]:
'''
[Nó de prática]

O código deste segmento foi dividido em 2 partes. Primeiramente, vamos aprender um dicionário de vocabulário para os dados de treinamento 
e então transformar os dados em uma matriz documento-termo; em seguida, para os dados de teste apenas transformaremos 
os dados em uma matriz documento-termo.

Isto é similar ao processo que seguimos no Passo 2.3

Nós forneceremos os dados transformados para os alunos através das variáveis 'training_data' e 'testing_data'.
'''

In [ ]:
'''
Solução
'''
# Instancie o método CountVectorizer method
count_vector = CountVectorizer()

# Ajuste os dados de treinamento e retorne a matriz
training_data = count_vector.fit_transform(X_train)

# Transforme dados de teste e retorne a matriz. Note que não estamos ajustando os dados de texto no CountVectorizer()
testing_data = count_vector.transform(X_test)

### Passo 4.1: Implementação do teorema de Bayes do zero ###

Agora que temos nosso dataset no formato que precisamos, podemos seguir em frente para a próxima porção da nossa missão que é o algoritmo que usaremos para fazer nossas previsões, classificando uma mensagem como spam ou não-spam. Lembre-se que no início da missão nós discutimos brevemente o teorema de Bates, mas agora vamos nos entrar em mais detalhes. Para leigos, o teorema de Bayes calcula a probabilidade de um evento ocorrer, baseado em algumas outras probabilidades relacionadas ao evento em questão. Ele é composto de conhecimento a priori (as probabilidades que conhecemos ou que são dadas) e a posteriori (as probabilidades que gostaríamos de calcular usando o conhecimento a priori). 

Vamos implementar o teorema de Bayes do zero usando um exemplos simples. Digamos  que estamos tentando procurar a chance de um indivíduo ter diabetes, dado que ele ou ela fez um exame que teve um resultado positivo - isto é, indicando que o paciente teria diabetes. No campo médico, tais probabilidades assumem um papel importante, uma vez que frequentemente se trata de uma situação de vida ou morte.

Presumimos o seguinte:

`P(D)` é a probabilidade de uma pessoa ter Diabetes. Seu valor é `0.01` ou, em outras palavras, 1% da população geral tem diabetes (Aviso: estes valores são premissas e não refletem nenhum estudo médico).

`P(Pos)` é a probabilidade de ter um resultado positivo do exame.

`P(Neg)` é a probabilidade de ter um resultado negativo do exame.

`P(Pos|D)` é a probabilidade de se ter um resultado positivo em um exame feito para detectar diabetes, dado que você tem diabetes. Seu valor é `0.9`. Em outras palavras, o exame está correto 90% do tempo. Isto também é chamado de Sensibilidade ou Taxa de Verdadeiros Positivos.

`P(Neg|~D)` é a probabilidade de se ter um resultado negativo em um exame feito para detectar diabetes, dado que você não tem diabetes. Seu valor também é `0.9` e o exame está correto, portanto, 90% do tempo. Isto também é chamado de Especificidade ou Taxa de Falsos Positivos.

A fórmula de Bayes é como a seguir:

<img src="images/bayes_formula.png" height="242" width="242">

* `P(A)` é a probabilidade a priori de A acontecer independentemente. No nosso exemplo, isto seria `P(D)`. Este valor é dado para nós.

* `P(B)` é a probabilidade a priori de B acontecer independentemente. No nosso exemplo, isto seria `P(Pos)`.

* `P(A|B)` é a probabilidade a posteriori probability de A acontecer dado que B aconteceu. No nosso exemplo, isto seria `P(D|Pos)`. Isto é, **a probabilidade de um indivíduo ter diabetes, dado que aquele indivíduo obteve um resultado positivo do exame. Este é o valor que gostaríamos de calcular.**

* `P(B|A)` é probabilidade de verossimilhança de B acontecer, dado A. No nosso exemplo, isto seria `P(Pos|D)`. Este valor é dado para nós.

Colocando nossos valores na fórmula do teorema de Bayes, obtemos:

`P(D|Pos) = (P(D) * P(Pos|D) / P(Pos)`

A probabilidade de se ter um resultado positivo do exame `P(Pos)` pode ser calculada usando a Sensibilidade e Especificidade como a seguir:

`P(Pos) = [P(D) * Sensitivity] + [P(~D) * (1-Specificity))]`

In [ ]:
'''
Instruções:
Calcule a probabilidade de se obter um resultado positivo no exame, P(Pos)
'''

In [ ]:
'''
Solução (um esqueleto de código será fornecido)
'''
# P(D)
p_diabetes = 0.01

# P(~D)
p_no_diabetes = 0.99

# Sensitivity or P(Pos|D)
p_pos_diabetes = 0.9

# Specificity or P(Neg/~D)
p_neg_no_diabetes = 0.9

# P(Pos)
p_pos = (p_diabetes * p_pos_diabetes) + (p_no_diabetes * (1 - p_neg_no_diabetes))
print('A probabilidade de se obter um resultado positivo do exame P(Pos) é: {}',format(p_pos))

** A partir de todas estas informações podemos calcular nossas probabilidades a posteriori como a seguir: **
    
A probabilidade de um indivíduo ter diabetes, dado que aquele indivíduo teve um resultado positivo no exame:

`P(D/Pos) = (P(D) * Sensitivity)) / P(Pos)`

A probabilidade de um indivíduo não ter diabetes, dado que aquele indivíduo teve um resultado positivo no exame:

`P(~D/Pos) = (P(~D) * (1-Specificity)) / P(Pos)`

A soma das nossas probabilidades a posteriori sempre será igual a `1`. 

In [ ]:
'''
Instruções:
Calcule a probabilidade de um indivíduo ter diabetes, dado que ele teve um resultado positivo no exame.
Em outras palavras, calcule P(D|Pos).

A fórmula é: P(D|Pos) = (P(D) * P(Pos|D) / P(Pos)
'''

In [ ]:
'''
Solução
'''
# P(D|Pos)
p_diabetes_pos = (p_diabetes * p_pos_diabetes) / p_pos
print('A probabilidade de um indivíduo ter diabetes, dado que ele teve um resultado positivo no exame é:\
',format(p_diabetes_pos)) 

In [ ]:
'''
Instruções:
Calcule a probabilidade de um indivíduo não ter diabetes, dado que ele teve um resultado positivo no exame.
Em outras palavras, calcule P(~D|Pos).

A fórmula é: P(~D|Pos) = (P(~D) * P(Pos|~D) / P(Pos)

Note que P(Pos/~D) pode ser calculada como 1 - P(Neg/~D). 

Portanto:
P(Pos/~D) = p_pos_no_diabetes = 1 - 0.9 = 0.1
'''

In [ ]:
'''
Solução
'''
# P(Pos/~D)
p_pos_no_diabetes = 0.1

# P(~D|Pos)
p_no_diabetes_pos = (p_no_diabetes * p_pos_no_diabetes) / p_pos
print 'A probabilidade de um indivíduo não ter diabetes, dado que ele teve um resultado positivo no exame é:'\
,p_no_diabetes_pos

Parabéns! Você implementou o teorema de Bayes do zero. Nossa análise mostra que mesmo que você obtenha um resultado positivo em um exame, existe apenas uma probabilidade de 8.3% que você realmente tenha diabetes e de 91.67% que você não tenha diabetes. Isto, é claro, presumindo que apenas 1% da população como um todo teria diabetes, o que foi apenas uma premissa que usamos para este exemplo.

** O que é que o termo "Ingênuo" em "Bayes Ingênuo" significa? ** 

O termo "Ingênuo" no Bayes Ingênuo vem do fato que o algoritmo considera que as características que utilizará para fazer previsões são independentes um do outro, o que nem sempre é verdade. Então no nosso exemplo da Diabetes, estamos considerando apenas uma característica, que é o resultado do exame. Digamos que adicionamos outra característica, "exercício". Digamos que esta característica tenha um valor binário, em que `0` significa que o indivíduo se exercita por 2 dias na semana ou menos e `1` significa que o indivíduo se exercita por 3 dias na semana ou mais. Se tivéssemos que usar ambas características para calcular nossas probabilidades finais, tanto o resultado do exame quanto o valor da característica "exercício", o teorema de Bayes falharia. O Bayes Ingênuo é uma extensão do teorema de Bayes que presume que todas as características são independentes umas das outras. 

### Passo 4.2: Implementação do Bayes Ingênuo do zero ###



Agora que você entendeu todos detalhes do teorema de Bayes, vamos extendê-lo para considerar casos em que você tem mais de uma característica. 

Digamos que temos dois candidatos políticos, 'Jill Stein' do Partido Verde e 'Gary Johnson' do Partido Libertário e temos as probabilidades de cada um destes candidatos dizerem as palavras 'freedom', 'immigration' e 'environment' quando fizerem um discurso:

* Probabilidade que Jill Stein diga 'freedom': 0.1 ---------> `P(F|J)`
* Probabilidade que Jill Stein diga 'immigration': 0.1 -----> `P(I|J)`
* Probabilidade que Jill Stein diga 'environment': 0.8 -----> `P(E|J)`


* Probabilidade que Gary Johnson diga 'freedom': 0.7 -------> `P(F|G)`
* Probabilidade que Gary Johnson diga 'immigration': 0.2 ---> `P(I|G)`
* Probabilidade que Gary Johnson diga 'environment': 0.1 ---> `P(E|G)`


E vamos também presumir que a probabilidade de Jill Stein fazer um discurso, `P(J)` seja `0.5` e o mesmo para Gary Johnson, `P(G) = 0.5`. 

Dado este contexto, como seria se quiséssemos encontrar as probabilidades de Jill Stein dizer as palavras 'freedom' e 'immigration'? É aí que entra o algoritmo ingênuo de Bayes, uma vez que estamos considerando duas características: 'freedom' e 'immigration'.

Agora chegamos a um ponto em que podemos definir a fórmula para o teorema ingênuo de Bayes:

<img src="images/naivebayes.png" height="342" width="342">

Aqui `y` é a variável de classe ou, no nosso caso, o nome do candidato e `x1` até `xn` são os vetores de características ou, no nosso caso, palavras possivelmente ditas em um discurso. O teorema parte da premissa que cada um dos vetores de características ou palavras (`xi`) são independentes dos demais.

Para separar este processo, precisamos calcular as seguintes probabilidades a posteriori:

* `P(J|F,I)`: Probabilidade que Jill Stein diga as palavras Freedom e Immigration. 

    Usando a fórmula e nosso conhecimento do teorema de Bayes, podemos calcular esta probabilidade como a seguir: `P(J|F,I)` = `(P(J) * P(F|J) * P(I|J)) / P(F,I)`. Aqui `P(F,I)` é a probabilidade que as palavras 'freedom' e 'immigration' sejam ditas em um discurso.
    

* `P(G|F,I)`: Probabilidade que Gary Johnson diga as palavras Freedom e Immigration.  
    
    Usando a fórmula, podemos calcular isto como a seguir: `P(G|F,I)` = `(P(G) * P(F|G) * P(I|G)) / P(F,I)`

In [ ]:
'''
Instruções: Calcule a probabilidade que as palavras 'freedom' e 'immigration' sejam ditas em um discurso, ou
P(F,I).

O primeiro passo é multiplicar as probabilidades que Jill Stein faça um discurso pela probabilidade 
dela dizer as palavras 'freedom' e 'immigration' individualmente. Armazene o resultado em uma variável chamada de p_j_text

O segundo passo é multiplicar as probabilidades que Gary Johnson faça um discurso pela sua probabilidade
dele dizer as palavras 'freedom' e 'immigration' individualmente. Armazene o resultado em uma variável chamada de p_g_text

O terceiro passo é somar ambas probabilidades, que resultará em P(F,I).
'''

In [ ]:
'''
Solução: Passo 1
'''
# P(J)
p_j = 0.5

# P(F/J)
p_j_f = 0.1

# P(I/J)
p_j_i = 0.1

p_j_text = p_j * p_j_f * p_j_i
print(p_j_text)

In [ ]:
'''
Solução: Passo 2
'''
# P(G)
p_g = 0.5

# P(F/G)
p_g_f = 0.7

# P(I/G)
p_g_i = 0.2

p_g_text = p_g * p_g_f * p_g_i
print(p_g_text)

In [ ]:
'''
Solução: Passo 3: Calcule P(F,I) e armazene em p_f_i
'''
p_f_i = p_j_text + p_g_text
print('A probabilidade das palavras freedom e immigration sejam ditas é: ', format(p_f_i))

Agora podemos calcular a probabilidade de `P(J|F,I)`, que é a probabilidade que Jill Stein diga as palavras Freedom e Immigration e `P(G|F,I)`, que é a probabilidade que Gary Johnson diga as palavras Freedom e Immigration.

In [ ]:
'''
Instruções:
Calcule P(J|F,I) usando a fórmula P(J|F,I) = (P(J) * P(F|J) * P(I|J)) / P(F,I) e armazene na variável p_j_fi
'''

In [ ]:
'''
Solução
'''
p_j_fi = p_j_text / p_f_i
print('A probabilidade que Jill Stein diga as palavras Freedom e Immigration: ', format(p_j_fi))

In [ ]:
'''
Instruções:
Calcule P(G|F,I) usando a fórmula P(G|F,I) = (P(G) * P(F|G) * P(I|G)) / P(F,I) e armazene na variável p_g_fi
'''

In [ ]:
'''
Solução
'''
p_g_fi = p_g_text / p_f_i
print('A probabilidade que Gary Johnson diga as palavras Freedom e Immigration: ', format(p_g_fi))

Como podemos ver, da mesma forma que aconteceu no caso do teorema de Bayes, a some das nossas probabilidades a posteriori é igual a 1. Parabéns! Você implementou o teorema ingênuo de Bayes do zero. nossa análise mostra que existe apenas 6,6% de chance que Jill Stein do Partido Verde use as palavras 'freedom' e 'immigration' no seu discurso, comparado a 93,3% de chance para Gary Johnson do Partido Libertário.

Outro exemplo mais genérico do Bayes Ingênuo em ação é quando procuramos pelo termo 'Sacramento Kings' em um mecanismo de busca. Para obtermos os resultados relativos ao time de basquete da NBA, o mecanismo de busca precisa ser capaz de associar as duas palavras conjuntamente e não tratá-las individualmente, em cujo caso obteríamos resultados de fotos rotuladas com 'Sacramento', como imagens da paisagem da cidade, ou imagens de 'Kings', que poderiam ser fotos de coroas ou reis da história, quando o que estamos realmente procurando são imagens de um time de basquete. Este é o caso clássico do mecanismo de busca tratar palavras como entidades independentes, o que dá o nome "ingênuo" a esta abordagem. 

Ao aplicar este método ao nosso problema de classificação de mensagens como spam, o algoritmo Ingênuo de Bayes *olha para cada palavra individualmente e não como entidades associadas* com qualquer tipo de elo entre elas. No caso de detectores de spam isto normalmente funciona, uma vez que existem certas palavras que quase garantem que a mensagem seja classificada como spam. Por exemplo, e-mails com palavras como 'viagra' são normalmente classificados como spam.

### Passo 5: Implementação do Bayes Ingênuo usando scikit-learn ###

Felizmente, o sklearn tem várias implementações do Bayes Ingênuo que podemos usar, de modo que não precisamos fazer todos os cálculos do zero. Vamos usar o método `sklearn.naive_bayes` do sklearn para fazer previsões no nosso dataset. 

Especificamente, usaremos a implementação multinomial do Bayes Ingênuo. Este classificador específico é adequado para classificação com características discretas (como no nosso caso, contagens de palavras para classificação de texto). Ele recebe as contagens inteiras de palavras como sua entrada. Do outro lado, o Bayes Ingênuo Gaussiano é mais adequado para dados contínuos, uma vez presume que dados de entrada não teriam uma distribuição Gauassiana (ou normal).

In [ ]:
'''
Instruções:

Carregamos os dados de treinamento na variável 'training_data' e os dados de teste na 
variável 'testing_data'.

Importe o classificador MultinomialNB e ajuste os dados de treinamento ao classificador usando fit(). Nomeie seu classificador
como 'naive_bayes'. Você treinará o classificador usando 'training_data' e y_train' da nossa divisão anterior. 
'''

In [ ]:
'''
Solução
'''
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

In [ ]:
'''
Instruções:
Agora que nosso algoritmo foi treinado usando o dataset de treinamento, podemos fazer algumas previsões nos dados de teste
armazenados em 'testing_data' usando predict(). Armazene suas previsões na variável 'predictions'.
'''

In [ ]:
'''
Solução
'''
predictions = naive_bayes.predict(testing_data)

Agora que as previsões foram feitas no nosso conjunto de teste, precisamos verificar a precisão das nossas previsões.

### Passo 6: Avaliando nosso modelo ###

Agora que fizemos previsões no nosso conjunto de teste, nosso próximo objetivo é avaliar quão bem nosso modelo está se saindo. Existem várias formas de fazer isso, mas vamos começar com uma revisão rápida delas.

** Acurácia ** mede quão frequentemente o classificador faz a previsão correta. É a razão entre o número de previsões corretas e o total de previsões (o número de pontos de dados de teste).

** Precisão ** nos diz qual a proporção das mensagens que classificamos como spam, que realmente eram spam.
É a razão entre os verdadeiros positivos (palavras classificadas como spam, e que realmente são spam) e todos positivos (todas palavras classificadas como spam, independentemente desta classificação ter sido correta), em outras palavras é a razão de

`[Verdadeiros Positivos/(Verdadeiros Positivos + Falsos Positivos)]`

** Recall (sensibilidade)** nos diz qual a proporção de mensagens que realmente eram spam que foram classificadas por nós como spam.
É a razão entre verdadeiros positivos (palavras classificadas como spam, e que realmente são spam) e todas palavras que realmente eram spam, em outras palavras é a razão de

`[Verdadeiros Positivos/(Verdadeiros Positivos + Falsos Negativos)]`

Para problemas de classificação que são enviesados na sua distribuição de classificações, como é o nosso caso (por exemplo se tivéssemos 100 mensagens de texto e apenas 2 fossem spam e as 98 outras não fossem), a acurácia por si só não é uma boa métrica. Classificaríamos 90 mensagens como não-spam (inclusive as 2 que eram spam, mas que classificaríamos erroneamente como não-spam, tornando-as assim falsos negativos) e 10 como spam (todas 10 falsos positivos) e ainda assim obteríamos um escore de acurácia razoável. Para tais casos, precisão e recall acabam sendo bem úteis. Combinando estas duas métricas, podemos obter o escore F1, que é a média ponderada dos escores de precisão e recall. Este escore varia de 0 a 1, sendo 1 o melhor escore F1 possível.

Vamos usar todas 4 métricas para garantir que nosso modelo se dê bem. Para todas 4 métricas cujos valores podem variar de 0 a 1, obter um escore tão próximo de 1 quanto possível é um bom indicador de quão bem nosso modelo está indo.

In [ ]:
'''
Instruções:
Calcule a acurácia, precisão, recall e escore F1 do seu modelo usando seus dados de teste 'y_test' e as previsões
que você fez anteriormente e armazenou na variável 'predictions'.
'''

In [ ]:
'''
Solução
'''
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Acurácia: ', format(accuracy_score(y_test, predictions)))
print('Precisão: ', format(precision_score(y_test, predictions)))
print('Recall: ', format(recall_score(y_test, predictions)))
print('Escore F1: ', format(f1_score(y_test, predictions)))

### Passo 7: Conclusão ###

Uma das maiores vantagens que o Bayes Ingênuo tem sobre os outros algoritmos de classificação é sua capacidade de lidar com um número de características extremamente grande. No nosso caso, cada palavra é tratada como uma característica e existem milhares de palavras diferentes. Além disso, ele funciona bem na presença de características irrelevantes, não sendo significativamente afetado por elas. Outra grande vantagem é sua simplicidade relativa. O Bayes Ingênuo funciona bem logo de cara, e ajustar seus parâmetros quase nunca é necessário, exceto em casos em que a distribuição dos dados é conhecida. 
Quase nunca há sobreajuste nos dados. Outra vantagem importante é que os tempos de treinamento e previsão do seu modelo são bem rápidos para a quantidade de dados com que pode lidar. No geral, o Bayes Ingênuo é um baita algoritmo!

Parabéns! Você projetou, com sucesso, um modelo capaz de prever eficientemente se uma mensagem SMS é spam ou não!

Obrigado por aprender conosco!